In [49]:
import os
import csv
import numpy as np
from matplotlib import pyplot as plt

In [50]:
def is_number(s):
    """ Returns True if string is a number. """
    try:
        float(s)
        return True
    except ValueError:
        return False

In [51]:
selected_dir = "../benchmarking_results/csv_results"

result_summary_files = []

# Search for the available json files in the results folder:
for subdir, dirs, files in os.walk(selected_dir):
    for file in files:
        if file.endswith(".csv"):
            result_summary_files.append(os.path.join(subdir, file))
    break # <-- This ensures that we only search in a depth of one.
result_summary_files.sort()
for file in result_summary_files:
    print(file.split("/")[-1])


netlib_benchmark_1.0e-4_PDLP+STSN_timing.csv
netlib_benchmark_1.0e-4_PDLP+calc_dual_sum_timing.csv
netlib_benchmark_1.0e-4_PDLP+calc_sim_sharded_timing.csv
netlib_benchmark_1.0e-4_PDLP+calc_sim_timing.csv
netlib_benchmark_1.0e-4_PDLP+use_input_method_timing.csv
netlib_benchmark_1.0e-4_PDLP_timing.csv


In [52]:
# Data for x-axis in plots: 
similarity_threshold_list = []
kappa_value_list = []
lambda_value_list = []

# Data for y-axis in plots: 
total_iterations_list = []
solved_instances_list = []

for file in result_summary_files:
    total_iterations = 0
    total_seconds = 0
    total_KKT_passes = 0
    instance_counter = 0
    solved_instances = 0
    total_num_errors = 0
    kkt_sgm_10 = 1
    # print(file.split("/")[-1].removesuffix(".csv"))
    similarity_threshold = file.split("threshold=")[-1].removesuffix(".csv")
    kappa=file.split("kappa=")[-1].split("_")[0]
    lambda_val=file.split("lambda=")[-1].split("_")[0]
    if not is_number(kappa):
        kappa = 0 
    if not is_number(lambda_val):
        lambda_val = 0
    if not is_number(similarity_threshold):
        similarity_threshold = "1.0"
    # print("Similarity Threshold: ", similarity_threshold)
    with open(file, "r") as csv_file:
        file_reader = csv.reader(csv_file)
        
        for line in file_reader:
            instance_counter += 1
        instance_counter -= 1
  
    with open(file, "r") as csv_file:
        file_reader = csv.reader(csv_file)
        
        first_row = True
        for line in file_reader: 
            if first_row:
                first_row = False
            else:
                termination_reason = line[1].strip()
                if termination_reason == "TERMINATION_REASON_OPTIMAL":
                    solved_instances += 1
                if termination_reason == "TERMINATION_REASON_NUMERICAL_ERROR":
                    total_num_errors += 1
                if termination_reason == "TEMINATION_REASON_ITERATION_LIMIT" or termination_reason == "TERMINATION_REASON_NUMERICAL_ERROR":
                    kkt_passes = 100000
                else:
                    kkt_passes = float(line[4].strip())
                total_iterations += int(line[2].strip())
                total_seconds += float(line[3].strip())
                total_KKT_passes += kkt_passes
                
                kkt_sgm_10 *= (kkt_passes + 10)**(1/instance_counter)
    kkt_sgm_10 = (kkt_sgm_10) - 10

    print(file.split("/")[-1])    
    # print(" - Number of instances solved: ", solved_instances, "/", instance_counter) 
    # print(" - Numerical errors: ", total_num_errors)
    print(" - Total seconds: ", total_seconds)
    print(" - Total iterations: ", total_iterations) 
    print(" - Total KKT passes: ", total_KKT_passes)
    # print(" - SGM10 KKT passes: ", kkt_sgm_10)
    # Saving the stats for a comparison plot:
    similarity_threshold_list.append(float(similarity_threshold))
    kappa_value_list.append(float(kappa))
    lambda_value_list.append(float(lambda_val))
    total_iterations_list.append(int(total_iterations))
    solved_instances_list.append(solved_instances)

netlib_benchmark_1.0e-4_PDLP+STSN_timing.csv
 - Total seconds:  142.19007028500002
 - Total iterations:  2139919
 - Total KKT passes:  2151463.0
netlib_benchmark_1.0e-4_PDLP+calc_dual_sum_timing.csv
 - Total seconds:  135.54594054599997
 - Total iterations:  2389670
 - Total KKT passes:  2410901.0
netlib_benchmark_1.0e-4_PDLP+calc_sim_sharded_timing.csv
 - Total seconds:  144.95367966400002
 - Total iterations:  2389670
 - Total KKT passes:  2410901.0
netlib_benchmark_1.0e-4_PDLP+calc_sim_timing.csv
 - Total seconds:  146.851560242
 - Total iterations:  2389670
 - Total KKT passes:  2410901.0
netlib_benchmark_1.0e-4_PDLP+use_input_method_timing.csv
 - Total seconds:  131.63757262699997
 - Total iterations:  2389670
 - Total KKT passes:  2410901.0
netlib_benchmark_1.0e-4_PDLP_timing.csv
 - Total seconds:  131.93448142
 - Total iterations:  2389670
 - Total KKT passes:  2410901.0
